<a href="https://colab.research.google.com/github/tlimaroberto/spark-stuffs/blob/main/grpby_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!apt update
!apt install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os

os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"]= "/content/spark-2.3.1-bin-hadoop2.7/"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       

In [18]:
!wget -O /tmp/challenge.csv 'https://github.com/tlimaroberto/spark-stuffs/blob/main/challenge.csv?raw=True'

--2021-01-07 00:42:10--  https://github.com/tlimaroberto/spark-stuffs/blob/main/challenge.csv?raw=True
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tlimaroberto/spark-stuffs/raw/main/challenge.csv [following]
--2021-01-07 00:42:10--  https://github.com/tlimaroberto/spark-stuffs/raw/main/challenge.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tlimaroberto/spark-stuffs/main/challenge.csv [following]
--2021-01-07 00:42:10--  https://raw.githubusercontent.com/tlimaroberto/spark-stuffs/main/challenge.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

In [19]:
df = spark.read.csv('/tmp/challenge.csv', header = True)

In [20]:
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [21]:
from pyspark.sql.functions import *
df = df.withColumn('is_Mexico', when(df.Country == 'Mexico','yes').otherwise('no'))

In [22]:
df.show()

+---------------+--------------+-----------------+----------+---------+
|     ip_address|       Country|      Domain Name|Bytes_used|is_Mexico|
+---------------+--------------+-----------------+----------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|       no|
| 119.239.207.13|         China|         youtu.be|        51|       no|
|  68.69.217.210|         China|        adobe.com|        10|       no|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|       no|
|   211.13.10.68|     Indonesia|          hud.gov|        29|       no|
|   239.80.21.97|      Suriname|       smh.com.au|       218|       no|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|       no|
| 127.242.24.138|         China| surveymonkey.com|       123|       no|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|       no|
|   237.54.11.63|         China|       amazon.com|        83|       no|
| 252.141.157.25|         Japan|      cornell.edu|       374|   

In [23]:
import pyspark.sql.functions as sqlfunct
df1 = df.groupby('is_Mexico').agg(sqlfunct.sum(df.Bytes_used).alias('Soma'))

In [24]:
df1.show()

+---------+--------+
|is_Mexico|    Soma|
+---------+--------+
|       no|508076.0|
|      yes|  6293.0|
+---------+--------+



In [26]:
df2 = df.groupby("Country").agg(sqlfunct.countDistinct(df.ip_address).alias('Total'))
df2.sort(col('Total').desc()).show()

+--------------+-----+
|       Country|Total|
+--------------+-----+
|         China|  172|
|     Indonesia|  114|
|   Philippines|   65|
|        Russia|   56|
|        Brazil|   35|
|        Poland|   31|
|        Sweden|   28|
|         Japan|   25|
|      Portugal|   23|
|Czech Republic|   23|
|        France|   21|
|          Peru|   19|
|      Colombia|   17|
| United States|   15|
|       Ukraine|   14|
|     Argentina|   14|
|        Mexico|   13|
|      Thailand|   12|
|       Nigeria|   11|
|        Canada|   11|
+--------------+-----+
only showing top 20 rows

